In [ ]:
%pylab inline
from google.colab import drive
drive.mount('/content/drive')

Populating the interactive namespace from numpy and matplotlib
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# install OpenCV
!pip install opencv-python

# the usual ...
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from skimage import img_as_float
import cv2
import math


In [ ]:
def GradientMag(img):
  img = img.astype('float32')
  blur = cv2.GaussianBlur(img, (3, 3), 0)
  gray = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
  sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5, scale=1, delta=0, borderType=cv2.BORDER_DEFAULT)
  sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5, scale=1, delta=0, borderType=cv2.BORDER_DEFAULT)
  
  energy = np.sqrt(sobelx**2+sobely**2)
  return energy

def Find_Seam(img, gradient):
  # Find seam with Dynamic programming
  # Returns min_col(the index at the last row w/ minimum sum of energy), and the index matrix
  rows, cols = gradient.shape
  # sum_energy is where we store the current sum of energy(DP method)
  sum_energy = np.zeros((rows, cols))
  # copy first row of energy matrix to sum_energy
  sum_energy[0, :] = gradient[0, :]
  # Index_mat store the previous pixel's column index in the path, first row all has values 0
  index_mat = np.zeros((rows, cols), dtype=object)

  # update sum_energy matrix after the 1st row:
  for i in range(1, rows):
    for j in range(cols):
      UpperM = sum_energy[i - 1, j]
      if j == 0:
        UpperL = float('inf')
        UpperR = sum_energy[i - 1, j + 1]
      elif j == cols - 1:
        UpperL = sum_energy[i - 1, j - 1]
        UpperR = float('inf')
      else:
        UpperL = sum_energy[i - 1, j - 1]
        UpperR = sum_energy[i - 1, j + 1]
      sum_energy[i, j] = gradient[i, j] + min(UpperL, UpperM, UpperR)
      # analyse
      if UpperL == min(UpperL, UpperM, UpperR):
        index_mat[i, j] = int(j - 1)
      elif UpperM == min(UpperL, UpperM, UpperR):
        index_mat[i, j] = int(j)
      else:
        index_mat[i, j] = int(j + 1)

  # print('index mat is,', index_mat[1])

  # Take last row and finding smallest sum of energy: 
  min_col = np.argmin(sum_energy[rows - 1])
  return min_col, index_mat


def Remove_Seam(img, min_col, index_mat):
  #Returns the image with the vertical seam removed according to index_mat, the 
  #index of the last row(min_col) 
  #print('removed col is, ',min_col)
  rows = img.shape[0]
  cols = img.shape[1]
  result_img = np.zeros((rows, cols-1, 3))

  #Backtrace
  for i in range(rows-1, -1, -1):   
    result_img[i,:,0] = np.delete(img[i,:,0], min_col) 
    result_img[i,:,1] = np.delete(img[i,:,1], min_col) 
    result_img[i,:,2] = np.delete(img[i,:,2], min_col) 
    #find next index to delete
    index = index_mat[i,min_col]
    min_col = index
    #print('removed col is, ',min_col)

  return result_img

In [ ]:
def Remove_Col(img, num_removed_col):
    img = img_as_float(img)
    # Return an img with num_removed_col columns removed
    for i in range(num_removed_col):
        Gxy = GradientMag(img)
        min_col, index_mat = Find_Seam(img, Gxy)
        # print('sum energy is',sum_energy)
        # print("iteration i and minIndex is", i , min_col)
        # img = Find_seam_path(img, min_col, sum_energy)
        removed_img = Remove_Seam(img, min_col, index_mat)
        img = removed_img
        print("result img's shape", img.shape)

    return img

def Remove_Row(img, num_removed_row):
    # Return an img with num_removed_row rows removed
    transposed_img = cv2.transpose(img)
    for i in range(num_removed_row):
        Gxy = GradientMag(transposed_img)
        min_col, sum_energy = Find_Seam(transposed_img, Gxy)
        removed_img = Remove_Seam(transposed_img, min_col, sum_energy)
        transposed_img = removed_img
        print("result img's shape", transposed_img.shape)
    result_img = cv2.transpose(transposed_img)
    return result_img


def Part2(img, desired_size, remove_col_first):
    # if remove_col_first == True, first remove col, then remove_row
    r1, c1 = img.shape[0:2]
    r2, c2 = desired_size

    img = img_as_float(img)
    if r1 == r2 and c1 != c2:
        # remove cols
        removed_cols = c1 - c2
        result_img = Remove_Col(img, removed_cols)
        plt.imshow(result_img)
        print("final image has a shape of ", result_img.shape)
    elif c1 == c2 and r1 != r2:
        # remove rows
        removed_rows = r1 - r2
        result_img = Remove_Row(img, removed_rows)
        plt.imshow(result_img)
        print("final image has a shape of ", result_img.shape)
    else:
        if remove_col_first:
            removed_cols = c1 - c2
            removed_rows = r1 - r2
            first_img = Remove_Col(img, removed_cols)
            second_img = Remove_Row(first_img, removed_rows)
            print("final image has a shape of ", second_img.shape)
            dst_list = [first_img, second_img]
            # plot the figure
            fig = plt.figure(figsize=(8, 8))
            columns = 2
            rows = 1
            for i in range(1, columns * rows + 1):
                fig.add_subplot(rows, columns, i)
                plt.imshow(dst_list[i - 1])
        else:
            # remove rows first
            removed_cols = c1 - c2
            removed_rows = r1 - r2
            first_img = Remove_Row(img, removed_rows)
            second_img = Remove_Col(first_img, removed_cols)
            print("final image has a shape of ", second_img.shape)
            dst_list = [first_img, second_img]
            # plot the figure
            fig = plt.figure(figsize=(8, 8))
            columns = 2
            rows = 1
            for i in range(1, columns * rows + 1):
                fig.add_subplot(rows, columns, i)
                plt.imshow(dst_list[i - 1])

In [ ]:
# read an image (make sure CavePainting.jpg is in the main folder in your Google Drive)
img = cv2.imread('/content/drive/My Drive/Colab Notebooks/Assignments/A2/ex1.jpg') # READS IN NUMPY ARRAY
#convert color
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
print(img.shape)
ex1_desired_size = (968,957)
ex2_desired_size = (861,1200)
ex3_desired_size = (870,1200)
desired_size = ex3_desired_size
Part2(img,desired_size,False)

In [ ]:
#5.Comparison with cropping and scaling
#ex1
scaling_img1 = cv2.resize(img, (957,968))
cropped_img1 = img[:,0:957]
#ex2
scaling_img2 = cv2.resize(img, (1200,861))
cropped_img2 = img[:,0:1200]
cropped_img2 = cropped_img2[0:861,:]
#ex3
scaling_img3 = cv2.resize(img, (1200,870))
cropped_img3 = img[0:870,:]
cropped_img3 = cropped_img3[:,0:1200]
dst_list = [scaling_img3 , cropped_img3]
print("scaling img's shape is, ",scaling_img3.shape)
print("cropped img's shape is, ",cropped_img3.shape)
#plot the figure
fig=plt.figure(figsize=(8, 8))
columns = 2
rows = 1
for i in range(1, columns*rows +1):
    fig.add_subplot(rows, columns, i)
    plt.imshow(dst_list[i-1])